In [199]:
import pandas as pd
import pymongo
import requests
from bs4 import BeautifulSoup
import time
import json
from collections import Counter

In [2]:
mc = pymongo.MongoClient()  # Connect to the MongoDB server using default settings
db = mc['chess_predictions']  # Use (or create) a database called 'chess_predictions'
players = db['players']  # Use (or create) a collection called 'players'

In [3]:
def get_player_profile(username):
    try:
        response = requests.get(f'https://api.chess.com/pub/player/{username}')
        return json.loads(response.content.decode('utf-8'))
    except:
        return [] 
        
def get_player_stats(username):
    try:
        response = requests.get(f'https://api.chess.com/pub/player/{username}/stats')
        return json.loads(response.content.decode('utf-8'))
    except:
        return []
        
def get_player_games(username):
    '''return a list of lists where each list contains the games played for each month January through May 2018'''
    
    months = ['01', '02', '03', '04', '05']
    month_games = []
    for month in months:
        try:
            response = requests.get(f'https://api.chess.com/pub/player/{username}/games/2018/{month}')
            month_games.append([json.loads(response.content.decode('utf-8'))])
        except:
            continue
    return month_games

def player_data_to_mongoDB(username, mongoDB_connection, database, collection):
    
    mc = mongoDB_connection
    
    #use/create a database
    db = mc[database]
    
    #use/create a collection
    collection = db[collection]
    
    #query Chess.com api for data
    profile = get_player_profile(username)
    stats = get_player_stats(username)
    games = get_player_games(username)
    
    #insert player data into database
    collection.insert_one({**profile,
                           **stats,
                           'games': games
                            })

def all_player_data_to_mongoDB(players, mongoDB_connection, database, collection, verbose=False):
    '''insert all player data into a mongoDB
       Params:
          players: list of player names
          mongoDB_connection: open connection to database ex. pymongo.MongoClient()
          database: name of database
          collection: name of collection
       Keyword Args:
          verbose: if True print player name after each insert, default is False
    '''
    for player in players:
        player_data_to_mongoDB(player, mongoDB_connection, database, collection)
        if verbose:
            print(player)

In [224]:
player_data_to_mongoDB('babu500',pymongo.MongoClient(), 'chess_predictions', 'players')

In [18]:
df = pd.DataFrame(list(db['players'].find()))
df.head()

,@id,_id,avatar,chess960_daily,chess_blitz,chess_bullet,chess_daily,chess_rapid,country,followers,games,joined,last_online,location,name,player_id,status,url,username
0,https://api.chess.com/pub/player/babu500,5afa039f098388ad5ff6a3a1,NaN,NaN,"{'last': {'rating': 777, 'date': 1525820389, '...",NaN,"{'last': {'rating': 1200, 'date': 1516320601, ...","{'last': {'rating': 974, 'date': 1522794278, '...",https://api.chess.com/pub/country/US,1,[[{'games': [{'url': 'https://www.chess.com/li...,1516059527,1526310229,NaN,NaN,42361082,premium,https://www.chess.com/member/babu500,babu500
1,https://api.chess.com/pub/player/0rlandomagic,5afa05ce098388c83c3f08fa,NaN,"{'last': {'rating': 1257, 'date': 1526259679, ...","{'last': {'rating': 1800, 'date': 1526089345, ...","{'last': {'rating': 2117, 'date': 1526323151, ...","{'last': {'rating': 1425, 'date': 1525869836, ...",NaN,https://api.chess.com/pub/country/US,13,[[{'games': [{'url': 'https://www.chess.com/da...,1515424234,1526322679,NaN,NaN,42121838,basic,https://www.chess.com/member/0rlandoMagic,0rlandomagic
2,https://api.chess.com/pub/player/0ldtower,5afa063e098388c83c3f08fc,NaN,NaN,NaN,NaN,NaN,NaN,https://api.chess.com/pub/country/US,0,"[[{'games': []}], [{'games': []}], [{'games': ...",1515958780,1526117639,NaN,Nick Chamberlain,42324950,basic,https://www.chess.com/member/0ldTower,0ldtower
3,https://api.chess.com/pub/player/0rlandomagic,5afa0640098388c83c3f08fd,NaN,"{'last': {'rating': 1257, 'date': 1526259679, ...","{'last': {'rating': 1800, 'date': 1526089345, ...","{'last': {'rating': 2117, 'date': 1526323151, ...","{'last': {'rating': 1425, 'date': 1525869836, ...",NaN,https://api.chess.com/pub/country/US,13,[[{'games': [{'url': 'https://www.chess.com/da...,1515424234,1526322679,NaN,NaN,42121838,basic,https://www.chess.com/member/0rlandoMagic,0rlandomagic
4,https://api.chess.com/pub/player/0xvyper,5afa0641098388c83c3f08fe,NaN,NaN,"{'last': {'rating': 705, 'date': 1526217305, '...",NaN,NaN,"{'last': {'rating': 870, 'date': 1516238541, '...",https://api.chess.com/pub/country/US,1,[[{'games': [{'url': 'https://www.chess.com/li...,1515610811,1526216631,NaN,Vyper Dev,42192576,basic,https://www.chess.com/member/0xVyper,0xvyper


In [251]:
def is_inactive(x):
    if x < 1525132800:
        return 1
    else:
        return 0
    
df['inactive'] = df['last_online'].apply(lambda x: is_inactive(x))

In [ ]:
df.loc[0, may_games] = games_in_a_month(games_column, player, '05')

In [261]:
df.describe()

,followers,joined,last_online,player_id,standard,daily,blitz,True,lightning,False,...,E96,E97,E98,E99,lose,bughousepartnerlose,inactive,has_name,has_location,has_avatar
count,1452.000000,1.452000e+03,1.452000e+03,1.452000e+03,1452.000000,1452.000000,1452.000000,1452.000000,1452.000000,1452.000000,...,1452.0,1452.0,1452.0,1452.0,1452.0,1452.0,1452.0,1452.000000,1452.000000,1452.000000
mean,2.256887,1.516096e+09,1.526288e+09,4.237491e+07,29.226584,14.048209,280.601928,373.033058,52.094353,2.938017,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.626033,0.049587,0.438017
std,3.463224,7.614851e+05,4.475759e+04,2.854518e+05,104.201544,32.998847,495.484706,637.412341,364.312978,25.112900,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.484022,0.217164,0.496314
min,0.000000,1.514767e+09,1.526116e+09,4.187949e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
25%,0.000000,1.515457e+09,1.526262e+09,4.213664e+07,0.000000,0.000000,0.000000,18.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
50%,1.000000,1.516081e+09,1.526304e+09,4.236701e+07,0.000000,1.000000,45.000000,129.500000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.000000
75%,3.000000,1.516749e+09,1.526323e+09,4.261724e+07,4.250000,12.000000,374.500000,502.250000,2.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,1.000000
max,35.000000,1.517442e+09,1.526337e+09,4.288449e+07,1357.000000,313.000000,4460.000000,10429.000000,9947.000000,830.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000


In [240]:
def init_columns(df):
    '''Add columns for all eco codes, rated games, time_class, rules
    Params:
        df: pandas dataframe
    '''
    labels = ['A00', 'A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09',
               'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19',
               'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29',
               'A30', 'A31', 'A32', 'A33', 'A34', 'A35', 'A36', 'A37', 'A38', 'A39',
               'A40', 'A41', 'A42', 'A43', 'A44', 'A45', 'A46', 'A47', 'A48', 'A49',
               'A50', 'A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59',
               'A60', 'A61', 'A62', 'A63', 'A64', 'A65', 'A66', 'A67', 'A68', 'A69',
               'A70', 'A71', 'A72', 'A73', 'A74', 'A75', 'A76', 'A77', 'A78', 'A79',
               'A80', 'A81', 'A82', 'A83', 'A84', 'A85', 'A86', 'A87', 'A88', 'A89',
               'A90', 'A91', 'A92', 'A93', 'A94', 'A95', 'A96', 'A97', 'A98', 'A99',
               'B00', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09',
               'B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B17', 'B18', 'B19',
               'B20', 'B21', 'B22', 'B23', 'B24', 'B25', 'B26', 'B27', 'B28', 'B29',
               'B30', 'B31', 'B32', 'B33', 'B34', 'B35', 'B36', 'B37', 'B38', 'B39',
               'B40', 'B41', 'B42', 'B43', 'B44', 'B45', 'B46', 'B47', 'B48', 'B49',
               'B50', 'B51', 'B52', 'B53', 'B54', 'B55', 'B56', 'B57', 'B58', 'B59',
               'B60', 'B61', 'B62', 'B63', 'B64', 'B65', 'B66', 'B67', 'B68', 'B69',
               'B70', 'B71', 'B72', 'B73', 'B74', 'B75', 'B76', 'B77', 'B78', 'B79',
               'B80', 'B81', 'B82', 'B83', 'B84', 'B85', 'B86', 'B87', 'B88', 'B89',
               'B90', 'B91', 'B92', 'B93', 'B94', 'B95', 'B96', 'B97', 'B98', 'B99',
               'C00', 'C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08', 'C09',
               'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19',
               'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29',
               'C30', 'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39',
               'C40', 'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'C49',
               'C50', 'C51', 'C52', 'C53', 'C54', 'C55', 'C56', 'C57', 'C58', 'C59',
               'C60', 'C61', 'C62', 'C63', 'C64', 'C65', 'C66', 'C67', 'C68', 'C69',
               'C70', 'C71', 'C72', 'C73', 'C74', 'C75', 'C76', 'C77', 'C78', 'C79',
               'C80', 'C81', 'C82', 'C83', 'C84', 'C85', 'C86', 'C87', 'C88', 'C89',
               'C90', 'C91', 'C92', 'C93', 'C94', 'C95', 'C96', 'C97', 'C98', 'C99',
               'D00', 'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09',
               'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'D19',
               'D20', 'D21', 'D22', 'D23', 'D24', 'D25', 'D26', 'D27', 'D28', 'D29',
               'D30', 'D31', 'D32', 'D33', 'D34', 'D35', 'D36', 'D37', 'D38', 'D39',
               'D40', 'D41', 'D42', 'D43', 'D44', 'D45', 'D46', 'D47', 'D48', 'D49',
               'D50', 'D51', 'D52', 'D53', 'D54', 'D55', 'D56', 'D57', 'D58', 'D59',
               'D60', 'D61', 'D62', 'D63', 'D64', 'D65', 'D66', 'D67', 'D68', 'D69',
               'D70', 'D71', 'D72', 'D73', 'D74', 'D75', 'D76', 'D77', 'D78', 'D79',
               'D80', 'D81', 'D82', 'D83', 'D84', 'D85', 'D86', 'D87', 'D88', 'D89',
               'D90', 'D91', 'D92', 'D93', 'D94', 'D95', 'D96', 'D97', 'D98', 'D99',
               'E00', 'E01', 'E02', 'E03', 'E04', 'E05', 'E06', 'E07', 'E08', 'E09',
               'E10', 'E11', 'E12', 'E13', 'E14', 'E15', 'E16', 'E17', 'E18', 'E19',
               'E20', 'E21', 'E22', 'E23', 'E24', 'E25', 'E26', 'E27', 'E28', 'E29',
               'E30', 'E31', 'E32', 'E33', 'E34', 'E35', 'E36', 'E37', 'E38', 'E39',
               'E40', 'E41', 'E42', 'E43', 'E44', 'E45', 'E46', 'E47', 'E48', 'E49',
               'E50', 'E51', 'E52', 'E53', 'E54', 'E55', 'E56', 'E57', 'E58', 'E59',
               'E60', 'E61', 'E62', 'E63', 'E64', 'E65', 'E66', 'E67', 'E68', 'E69',
               'E70', 'E71', 'E72', 'E73', 'E74', 'E75', 'E76', 'E77', 'E78', 'E79',
               'E80', 'E81', 'E82', 'E83', 'E84', 'E85', 'E86', 'E87', 'E88', 'E89',
               'E90', 'E91', 'E92', 'E93', 'E94', 'E95', 'E96', 'E97', 'E98', 'E99',
               'win', 'checkmated', 'agreed', 'repetition', 'timeout', 'resigned', 
               'stalemate', 'lose', 'insufficient', '50move', 'abandoned', 'kingofthehill',
               'threecheck', 'timevsinsufficient', 'bughousepartnerlose', 'False', 'True',
               'standard', 'daily', 'blitz', 'lightning']
    for label in labels:
        df[label] = 0

In [255]:
def has_attribute(x):
    if x != x:
        return 0
    else:
        return 1
    
df['has_name'] = df['name'].apply(lambda x: has_attribute(x))
df['has_location'] = df['location'].apply(lambda x: has_attribute(x))
df['has_avatar'] = df['avatar'].apply(lambda x: has_attribute(x))

In [ ]:
#get data from mongoDB and put into dataframe
df = pd.DataFrame(list(db['players'].find()))

#remove duplicate users by player_id
df.drop_duplicates(subset='player_id', inplace=True)

#reset the index
df.reset_index(drop=True, inplace=True)

#create inactive column (1 = inactive, 0 = active) inactive if they have not logged in for two weeks (may 1st)
df['inactive'] = df['last_online'] < 1525132800


#create columns has_name, has_location, status, has_avatar


def has_attribute(x):
    #checks if NaN
    if x != x:
        return 0
    else:
        return 1
    
df['has_name'] = df['name'].apply(lambda x: has_attribute(x))
df['has_location'] = df['location'].apply(lambda x: has_attribute(x))
df['has_avatar'] = df['avatar'].apply(lambda x: has_attribute(x))

df['games_in_may'] = df[''].apply(lambda x: games_in_a_month(df['games'], x, '05'))

#remove unwanted columns
df.drop(columns=['@id', '_id', 'chess960_daily', 'chess_blitz', 'chess_bullet', 'chess_daily', 'chess_rapid', 'url'])



In [241]:
init_columns(df)

In [259]:
parse_games(df)

In [248]:
for label in df.columns:
    print(label)

@id
_id
avatar
chess960_daily
chess_blitz
chess_bullet
chess_daily
chess_rapid
country
followers
games
joined
last_online
location
name
player_id
status
url
username
standard
daily
blitz
True
lightning
False
chess
chess960
crazyhouse
kingofthehill
threecheck
bughouse
C50
C24
C48
C47
C25
C46
C40
D04
C11
D35
B02
C30
C20
B15
A01
D00
C42
B30
D07
C44
A40
D02
A15
A45
A00
E20
D43
B00
D31
C62
A41
B22
B01
C45
B06
C64
B21
A20
A43
D30
B32
B33
C00
D06
A53
A50
A03
B95
B29
B54
B10
B34
D10
A04
A08
E65
C21
D20
C65
A06
B90
B40
C41
B07
B50
B23
A30
A10
C60
C57
C23
C22
C26
D01
B20
C34
C55
C58
B27
C70
C33
C68
C61
C01
A21
C77
A46
D11
A02
A48
B28
C31
C43
C36
B13
C53
A05
B12
A80
A11
C02
B18
C59
B19
C51
C10
A07
B44
E61
D37
C13
E10
C18
E53
C78
A56
B75
B53
B97
C88
E92
E60
D08
E40
A57
B03
A13
A09
B08
C28
A27
A84
A25
D03
B52
B94
A36
A16
B45
A17
A85
A22
A34
C37
B51
B70
A35
C66
B43
B46
B16
A18
C32
A47
C63
C67
B56
B92
A28
D21
B86
C52
A91
E76
E70
E90
A52
A29
B72
B24
D05
A81
C15
D53
A82
E00
E81
E91
A49
A31
C74
D24
D50


In [254]:
df.last_online.min()

1526116025

In [225]:
def games_in_a_month(games_column, player, month):
    return len(games_column[player][month][0]['games'])
    
def rated_games(games_column, player, month, game):
    return str(games_column[player][month][0]['games'][game]['rated'])

def time_class_games(games_column, player, month, game):
    return games_column[player][month][0]['games'][game]['time_class']

def rules_games(games_column, player, month, game):
    return games_column[player][month][0]['games'][game]['rules']

def eco_games(games_column, player, month, game):
    return games_column[player][month][0]['games'][game]['eco'][31:].split('-')[0]

def results_games(games_column, player, month, game):
    if games_column[player][month][0]['games'][game]['white'] == df.username[0]:
        return games_column[player][month][0]['games'][game]['white']['result']
    else:
        return games_column[player][month][0]['games'][game]['black']['result']

def make_columns(features, df, index):
    '''Counts the appearances of each type of outcome then creates a column in the dataframe corresponding to
    that outcome and filling in the number of occurrences for each player
    Params:
        features: List of each outcome ex: [c, a, a, d, d, a, d, c, j]
        df: pandas dataframe
        index: index to located each player
    '''
        counters = Counter()
        for feature in features:
            counters[feature] += 1

        for counter in counters:
            df.loc[index, counter] = counters[counter]


def parse_games(df, column='games', verbose=False):
    '''Pull out stats from the column containing games and add columns to the datafram inplace for the stats
    Params:
        df: pandas dataframe
    Keyword Args:
        column: column containing games, default column name is 'games'
        verbose: if True, print player indices as they are parsed, default is False
    '''
    games_column = df[column]
    player_idx = range(df.shape[0])
    for player in player_idx:
        if verbose:
            print(player)
        rated = []
        time_class = []
        rules = []
        eco = []
        results = []
        if len(games_column[player]) != 5:
            continue
        else:
            for month in range(4):
                try:
                    for game in range(games_in_a_month(games_column, player, month)):
                        try:
                            rated.append(rated_games(games_column, player, month, game))
                        except KeyError:
                            continue
                        try:
                            time_class.append(time_class_games(games_column, player, month, game))
                        except KeyError:
                            continue
                        try:
                            rules.append(rules_games(games_column, player, month, game))
                        except KeyError:
                            continue
                        try:
                            eco.append(eco_games(games_column, player, month, game))
                        except KeyError:
                            continue
                        try:
                            results.append(results_games(games_column, player, month, game))
                        except KeyError:
                            continue
                except KeyError:
                    continue
                    
            df.loc[player, may_games] = games_in_a_month(games_column, player, '05')
            
            make_columns(time_class, df, player)
            make_columns(rated, df, player)
            make_columns(rules, df, player)
            make_columns(eco, df, player)
            make_columns(results, df, player)
            


In [135]:
df.drop(labels=['standard', 'daily', 'blitz', 'lightning'],axis=1, inplace=True)

In [137]:
df.drop(labels=['True', 'False'],axis=1, inplace=True)

In [ ]:
df['games'][player][month][0]['games'][one game from the month]['url',
                                                                'pgn',
                                                                'time_control',
                                                                'end_time',
                                                                'rated',
                                                                'fen',
                                                                'time_class',
                                                                'rules',
                                                                'white',
                                                                'black']


In [182]:
eco = []
for game in range(len(games[1][0][0]['games'])):
    try:
        eco.append(games[0][0][0]['games'][game]['eco'][31:].split('-')[0])
    except:
        continue
print(eco)

['C50', 'C24', 'C48', 'C47', 'C25', 'C47', 'C48', 'C46', 'C50']


In [206]:
games[0][0][0]['games'][0]['pgn'].split('} ')[-1]

'0-1'

In [213]:
def results():
    if games[0][0][0]['games'][0]['white'] == df.username[0]:
        return games[0][0][0]['games'][0]['white']['result']
    else:
        return games[0][0][0]['games'][0]['black']['result']

{'@id': 'https://api.chess.com/pub/player/babu500',
 'rating': 953,
 'result': 'checkmated',
 'username': 'babu500'}

In [214]:
df.username[0]

'babu500'

In [215]:
games[0][0][0]['games'][0]['black']['result']

'win'

In [228]:
games[0][0][0]['games'][0]['white']

{'@id': 'https://api.chess.com/pub/player/babu500',
 'rating': 953,
 'result': 'checkmated',
 'username': 'babu500'}

In [232]:
letters = ['A', 'B', 'C', 'D', 'E']
nums = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
eco_codes = []
for letter in letters:
    for num in nums:
        for num2 in nums:
            eco_codes.append(letter + num + num2)
print(eco_codes)

['A00', 'A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'A31', 'A32', 'A33', 'A34', 'A35', 'A36', 'A37', 'A38', 'A39', 'A40', 'A41', 'A42', 'A43', 'A44', 'A45', 'A46', 'A47', 'A48', 'A49', 'A50', 'A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A60', 'A61', 'A62', 'A63', 'A64', 'A65', 'A66', 'A67', 'A68', 'A69', 'A70', 'A71', 'A72', 'A73', 'A74', 'A75', 'A76', 'A77', 'A78', 'A79', 'A80', 'A81', 'A82', 'A83', 'A84', 'A85', 'A86', 'A87', 'A88', 'A89', 'A90', 'A91', 'A92', 'A93', 'A94', 'A95', 'A96', 'A97', 'A98', 'A99', 'B00', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B17', 'B18', 'B19', 'B20', 'B21', 'B22', 'B23', 'B24', 'B25', 'B26', 'B27', 'B28', 'B29', 'B30', 'B31', 'B32', 'B33', 'B34', 'B35', 'B36', 'B37', 'B38', 'B39', 'B40', 'B41', 'B42'